In [75]:
import re
import os

class DictionaryHandler:
    def add_to_dict(self, assignment_input, assignments):
        key, value = assignment_input.replace(" ", "").split("=")

        assignments[key] = value
        return assignments
        
class InputHandler:
    def __init__(self):
        self.assignment_pattern = re.compile(r'^[a-zA-Z0-9+\-*/**()= ]+$')

    def is_valid_assignment(self, assignment_string):
        if not assignment_string:
            print("Input Error: Assignment cannot be empty")
            return False
        elif not self.assignment_pattern.match(assignment_string):
            print("Input Error: Use of Invalid characters")
            return False
        elif '=' not in assignment_string:
            print("Input Error: Assignment must contain '='")
            return False
        else:
            return True
        
    def is_valid_filename(self, filename):
        if not filename:
            print("Filename cannot be empty")
            return False

        if not os.path.basename(filename) == filename:
            print("File cannot be fun")
            return False

        if '.' not in filename:
            filename += '.txt'

        return True
 

class EquationSorter:
    def __init__(self):
        self.eq_dict = {}
        self.dependency_graph = {}

    def _add_dependencies(self, var, sorted_eqs, seen, undefined_vars):
        if var in seen:
            raise ValueError(f"Circular dependency detected involving '{var}'")
        if var not in sorted_eqs:
            if var not in self.dependency_graph:
                undefined_vars.add(var)
                return
            seen.add(var)
            for dep in self.dependency_graph.get(var, []):
                self._add_dependencies(dep, sorted_eqs, seen, undefined_vars)
            sorted_eqs[var] = self.eq_dict[var]
            seen.remove(var)

    def sort_equations(self, equations):
        # parsing the equations to extract variables and their dependencies
        for eq in equations:
            var, expr = eq.split(" = ")
            deps = set([x for x in expr.split(" ") if x.isalpha() and x != var])
            self.eq_dict[var] = expr
            self.dependency_graph[var] = deps

        # sorting the equations based on dependencies
        sorted_eqs = {}
        seen = set()
        undefined_vars = set()
        for var in self.dependency_graph:
            self._add_dependencies(var, sorted_eqs, seen, undefined_vars)

        # adding undefined variables at the end
        for var in undefined_vars:
            sorted_eqs[var] = None

        return sorted_eqs

In [76]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        # define criteria for variable
        return char.isalpha()

    def _isoperator(self, char):
        # define criteria for operator
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or self._isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if (exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        tokens.append('(')
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            elif self._isvariable(exp[i]):
                var_start = i
                while i < len(exp) and self._isvariable(exp[i]):
                    i += 1
                tokens.append(exp[var_start:i])
                continue  # Ensure we skip the next else block after accumulating a variable
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(',expression.replace(" ", ""),')'))
        # print(tokenize_inner(exp)) #debug
        return self._tokenize_inner(exp)

In [77]:
expression = "a=(-lemon+2)"
print(''.join(('(',expression.replace(" ", ""),')')))

(a=(-lemon+2))


In [78]:
tkn = ExpressionTokenizer()

tkn.tokenize("a=(-lemon+2)")

['(', 'a', '=', '(', '(', 'l', '*', '-1', ')', 'emon', '+', '2', ')', ')']

In [93]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        # define criteria for variable
        return char.isalpha()

    def _isoperator(self, char):
        # define criteria for operator
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = ''
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or self._isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if self._isvariable(exp[i + 1]):
                        variable_name = ''
                        while i + 1 < len(exp) and self._isvariable(exp[i + 1]):
                            variable_name += exp[i + 1]
                            i += 1
                        tokens.extend(['(', variable_name, '*', '-1', ')'])
                        i += 1
                    elif (exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        tokens.append('(')
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)
    
tkn = ExpressionTokenizer()
tokens = tkn.tokenize("-(lemon**20)")
print(tokens)

['(', '(', '(', 'lemon', '**', '2', '0', ')', '*', '-1', ')', ')']


In [105]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                if i + 1 < len(exp) and (exp[i + 1].isdigit() or self._isvariable(exp[i + 1]) or exp[i + 1] == '('):
                    if exp[i + 1] == '(':
                        j = i + 2
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                            j += 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j-1])
                        tokens.extend(['(', *inside_tokens, '*', '-1', ')'])  # Correct placement of ')'
                        i = j  # Adjust to move past the processed section
                    elif exp[i + 1].isdigit():  # Negative number
                        number = exp[i + 1]
                        i += 2
                        while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                            number += exp[i]
                            i += 1
                        tokens.extend(['(', number, '*', '-1', ')'])
                    else:  # Negative variable
                        variable_name = exp[i + 1]
                        i += 2
                        while i < len(exp) and self._isvariable(exp[i]):
                            variable_name += exp[i]
                            i += 1
                        tokens.extend(['(', variable_name, '*', '-1', ')'])
                else:
                    tokens.append(exp[i])
                    i += 1
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)

tkn = ExpressionTokenizer()
tokens = tkn.tokenize("-(-lemon**-20)")
print(tokens)


['(', '(', '(', 'lemon', '*', '-1', ')', '**', '(', '20', '*', '-1', ')', '*', '-1', ')', ')']


In [100]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                if i + 1 < len(exp) and (exp[i + 1].isdigit() or self._isvariable(exp[i + 1])):
                    # Handle negative numbers and variables specifically
                    i += 1
                    if exp[i].isdigit():  # Negative number
                        number = exp[i]
                        i += 1
                        while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                            number += exp[i]
                            i += 1
                        tokens.extend(['(', number, '*', '-1', ')'])
                    else:  # Negative variable
                        variable_name = exp[i]
                        i += 1
                        while i < len(exp) and self._isvariable(exp[i]):
                            variable_name += exp[i]
                            i += 1
                        tokens.extend(['(', variable_name, '*', '-1', ')'])
                else:
                    tokens.append(exp[i])
                    i += 1
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)
    
tkn = ExpressionTokenizer()
tokens = tkn.tokenize("-(-lemon+-20)")
print(tokens)


['(', '-', '(', '(', 'lemon', '*', '-1', ')', '+', '(', '20', '*', '-1', ')', ')', ')']
